In [1]:
from datasets import load_dataset
import pandas as pd
from collections import Counter
import requests
from pprint import pprint
import json
import random
import multiprocessing
import itertools
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import precision_score, recall_score, f1_score

import torch
import torch.nn as nn #parent
import torch.nn.functional as F #activation func, max pooling funcs
from torch.utils.data import ConcatDataset
from torch.nn.parallel import DataParallel
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms


import math
import numpy as np

In [ ]:
API_URL = "https://llm.ispras.ru/api/chat/completions"
API_MODEL_URL = "https://llm.ispras.ru/api/models"
API_KEY = "YOUR_TOKEN"
with open('./secrets') as file:
    data: dict = json.load(file)
    API_KEY = data.get('API_KEY', 'FAILED TO LOAD')